# drifter processing step 0: from netcdf to pairs 

In [2]:
import os
from glob import glob
import pickle

import numpy as np
import xarray as xr
import dask.bag as db
#import pandas as pd
#import geopandas
#import datetime

from matplotlib import pyplot as plt
%matplotlib inline
#import cartopy.crs as ccrs
#import cartopy.feature as cfeature

import mitequinox.drifters as edr

data_dir = '/work/ALT/swot/aval/syn/drifters/'

In [1]:
from dask_jobqueue import PBSCluster
#cluster = PBSCluster(cores=24, processes=12, walltime='12:00:00') #, memory='20GB')
cluster = PBSCluster(cores=12, processes=6, walltime='12:00:00') #, memory='20GB')
w = cluster.scale(40)
#w = cluster.scale(10) # postprocessing

/home/mp/pontea/.conda/envs/equinox/lib/python3.6/site-packages/dask_jobqueue/config.py:12: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
/home/mp/pontea/.conda/envs/equinox/lib/python3.6/site-packages/distributed/deploy/local.py:138: UserWarning: diagnostics_port has been deprecated. Please use `dashboard_address=` instead
  "diagnostics_port has been deprecated. "


In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)
#client = Client()

In [4]:
#client
cluster

In [5]:
#client.close()
cluster.close()

Unknown job_id: 5313904 for worker tcp://10.135.36.90:33191
Unknown job_id: 5313902 for worker tcp://10.135.36.90:34790
Unknown job_id: 5313903 for worker tcp://10.135.36.90:42067
Unknown job_id: 5313905 for worker tcp://10.135.36.92:44369
Unknown job_id: 5313895 for worker tcp://10.135.36.85:38318
Unknown job_id: 5313894 for worker tcp://10.135.36.85:45128
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client


---
## utils

In [5]:
def load_trajectory(ij, ncfile, gps):
    ds = xr.open_dataset(ncfile).isel(TIME=slice(ij[0],ij[1]-1))
    ds['GPS'] = (gps + ds.U*0.).astype(int)
    df = ds.to_dataframe()
    id = int(df['ID'][0])
    df = df.drop(columns='ID')
    return df, id

def store(d, gps):
    if gps==1:
        file = data_dir+'single/gps_%09d.p' %d[1]
    else:
        file = data_dir+'single/argos_%09d.p' %d[1]        
    pickle.dump( d, open( file, 'wb' ) , protocol=-1)

---

## rewrite hourly data

### inspect single file

In [6]:
ncfile = data_dir+'raw/driftertrajWMLE_1.02_block1.nc'
ds = xr.open_dataset(ncfile)
print(ds)
print(ds.note)

<xarray.Dataset>
Dimensions:  (TIME: 15230718)
Coordinates:
  * TIME     (TIME) datetime64[ns] 2005-04-15T20:00:00 ... NaT
Data variables:
    ID       (TIME) float64 ...
    LAT      (TIME) float64 ...
    LON      (TIME) float64 ...
    U        (TIME) float64 ...
    V        (TIME) float64 ...
    LAT_ERR  (TIME) float64 ...
    LON_ERR  (TIME) float64 ...
    U_ERR    (TIME) float64 ...
    V_ERR    (TIME) float64 ...
    GAP      (TIME) timedelta64[ns] ...
    RMSGAP   (TIME) timedelta64[ns] ...
    DROGUE   (TIME) float64 ...
Attributes:
    title:        Hourly Argos-tracked drifters location and velocity estimat...
    description:  This is version 1.02, block 1 of the dataset. See http://ww...
    note:         For all variables of dimension TIME, interruptions in the e...
    creator:      Shane Elipot and Rick Lumpkin
    timestamp:    19-Dec-2018 13:25:49
For all variables of dimension TIME, interruptions in the estimation along a single trajectory are indicated by "Inf" v

In [8]:
inan = np.where(np.isnan(ds.LAT.values))[0]
b = (db.from_sequence([(i+1,j) for i,j in zip(np.hstack([-1,inan])[:-1],inan)], 
                      npartitions=1000)
     .map(load_trajectory, ncfile))
b.take(1)[0][0].head()

,LAT,LON,U,V,LAT_ERR,LON_ERR,U_ERR,V_ERR,GAP,RMSGAP,DROGUE
TIME,,,,,,,,,,,
2005-04-15 20:00:00,47.47501,312.05118,-0.1733,0.2009,0.01129,0.04254,0.0,0.0,00:54:43.200000,01:08:24,1.0
2005-04-15 21:00:00,47.46839,312.04200,-0.1616,-0.5554,0.00987,0.01140,0.0,0.0,01:04:48,00:58:12,1.0
2005-04-15 22:00:00,47.44858,312.03870,0.8653,-0.5284,0.00537,0.01501,0.0,0.0,00:33:07.200000,00:47:38.400000,1.0
2005-04-15 23:00:00,47.43282,312.07433,0.4313,-0.4776,0.00058,0.00803,0.0,0.0,00:28:48,00:49:33.600000,1.0
2005-04-16 00:00:00,47.41716,312.07013,0.0575,-0.4887,0.00049,0.00729,0.0,0.0,01:12:00,00:57:18,1.0


### process all nc block files

In [7]:
gps=1

ncfile = data_dir+'raw/driftertrajGPS_1.02.nc'
#
ds = xr.open_dataset(ncfile)
inan = np.where(np.isnan(ds.LAT.values))[0]
#
b = (db.from_sequence([(i+1,j) for i,j in zip(np.hstack([-1,inan])[:-1],inan)], 
                      npartitions=1000)
     .map(load_trajectory, ncfile, gps))
b.map(store, gps).compute()
print(ncfile)

/work/ALT/swot/aval/syn/drifters/raw/driftertrajGPS_1.02.nc


In [8]:
gps=0
for ifile in range(1,8):
    ncfile = data_dir+'raw/driftertrajWMLE_1.02_block%d.nc' %ifile
    #
    ds = xr.open_dataset(ncfile)
    inan = np.where(np.isnan(ds.LAT.values))[0]
    #
    b = (db.from_sequence([(i+1,j) for i,j in zip(np.hstack([-1,inan])[:-1],inan)], 
                          npartitions=1000)
         .map(load_trajectory, ncfile, gps))
    b.map(store, gps).compute()
    print(ncfile)

/work/ALT/swot/aval/syn/drifters/raw/driftertrajWMLE_1.02_block1.nc
/work/ALT/swot/aval/syn/drifters/raw/driftertrajWMLE_1.02_block2.nc
/work/ALT/swot/aval/syn/drifters/raw/driftertrajWMLE_1.02_block3.nc
/work/ALT/swot/aval/syn/drifters/raw/driftertrajWMLE_1.02_block4.nc
/work/ALT/swot/aval/syn/drifters/raw/driftertrajWMLE_1.02_block5.nc
/work/ALT/swot/aval/syn/drifters/raw/driftertrajWMLE_1.02_block6.nc
/work/ALT/swot/aval/syn/drifters/raw/driftertrajWMLE_1.02_block7.nc


In [9]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client


---

## reload files and search for temporal and geographical proximity

The parameter controling which pair will be selected is a distance `r_threshold`: any pair closer than this distance will be selected

In [6]:
r_threshold=500 # km

In [7]:
files = sorted(glob(data_dir+'single/*.p'))
#files = files[:500]
print('%d single trajectories' %len(files))

15329 single trajectories


In [8]:
def cross_traj(d1, iD, r_threshold):
    D = [pickle.load(open(f, 'rb')) for f in files[iD]]
    out = []
    for d in D:
        d2 = d[0]
        try:
            df = d1[0].join(d2, how='inner',lsuffix='_l',rsuffix='_r')
            if not df.empty:
                # compute distance
                r = edr.haversine(df['LON_l'],df['LAT_l'], 
                              df['LON_r'],df['LAT_r'])
                if not df[r<r_threshold].empty:
                    out.append([d[1],d1[1]])
        except:
            out.append([d[1],d1[1],'error'])
    if out:
        return out

### profile with few pairs

In [11]:
Nc = 10
i=0
iD = slice(i, min(i+Nc,len(files)))
%time b = ( db.from_sequence(files[:], npartitions=1000) \
              .map(lambda f: pickle.load(open(f, 'rb'))) )
%time lb = b.map(cross_traj, iD, r_threshold)
%time p = lb.compute()

CPU times: user 25.5 ms, sys: 2.95 ms, total: 28.5 ms
Wall time: 25.9 ms
CPU times: user 101 ms, sys: 3.94 ms, total: 105 ms
Wall time: 103 ms
CPU times: user 1min 27s, sys: 3.52 s, total: 1min 31s
Wall time: 4min 12s


In [13]:
iD = slice(i, min(i+Nc,len(files)))
b = ( db.from_sequence(files[:],npartitions=100) \
              .map(lambda f: pickle.load(open(f, 'rb'))) )
lb = b.map(cross_traj, iD, r_threshold)
%time p = lb.compute()

CPU times: user 21.6 ms, sys: 29 µs, total: 21.6 ms
Wall time: 18.9 ms
CPU times: user 9.3 ms, sys: 1.04 ms, total: 10.3 ms
Wall time: 8.49 ms
CPU times: user 12.6 s, sys: 520 ms, total: 13.2 s
Wall time: 1min 1s


Best answer:

In [15]:
Nc = 100
iD = slice(i, min(i+Nc,len(files)))
b = ( db.from_sequence(files[:],npartitions=100) \
              .map(lambda f: pickle.load(open(f, 'rb'))) )
lb = b.map(cross_traj, iD, r_threshold)
%time p = lb.compute()

CPU times: user 85.1 ms, sys: 2.03 ms, total: 87.1 ms
Wall time: 83.1 ms
CPU times: user 12.5 ms, sys: 0 ns, total: 12.5 ms
Wall time: 10.7 ms
CPU times: user 1min 45s, sys: 4.03 s, total: 1min 49s
Wall time: 8min 27s


### process all pairs

In [12]:
# loop around groups of drifters and
Nc = 100
i0 = 8100 # historic restart at 730
for i in range(i0,len(files),Nc):
    
    file = data_dir+'pairs/%09d.p' %i
    
    if not os.path.isfile(file):

        iD = slice(i, min(i+Nc,len(files)))

        # load dask bag
        b = ( db.from_sequence(files[i+1:], npartitions=100)
              .map(lambda f: pickle.load(open(f, 'rb'))) )

        # cross data
        p = b.map(cross_traj, iD, r_threshold).compute()
        p = [llp for lp in p if lp is not None for llp in lp]
        p = [lp for lp in p if (lp[0]!=lp[1])]

        pickle.dump( p, open( file, 'wb' ) , protocol=-1)
        
        #client.cancel(b)
        #del b, p
    
    print('%d / %d'%(i,len(files)))

8100 / 15329
8200 / 15329
8300 / 15329
8400 / 15329
8500 / 15329
8600 / 15329
8700 / 15329
8800 / 15329
8900 / 15329
9000 / 15329
9100 / 15329
9200 / 15329
9300 / 15329
9400 / 15329
9500 / 15329
9600 / 15329
9700 / 15329
9800 / 15329
9900 / 15329
10000 / 15329
10100 / 15329
10200 / 15329
10300 / 15329
10400 / 15329
10500 / 15329
10600 / 15329
10700 / 15329
10800 / 15329
10900 / 15329
11000 / 15329
11100 / 15329
11200 / 15329
11300 / 15329
11400 / 15329
11500 / 15329
11600 / 15329
11700 / 15329
11800 / 15329
11900 / 15329
12000 / 15329
12100 / 15329
12200 / 15329
12300 / 15329
12400 / 15329
12500 / 15329
12600 / 15329
12700 / 15329
12800 / 15329
12900 / 15329
13000 / 15329
13100 / 15329
13200 / 15329
13300 / 15329
13400 / 15329
13500 / 15329
13600 / 15329
13700 / 15329
13800 / 15329
13900 / 15329
14000 / 15329
14100 / 15329
14200 / 15329
14300 / 15329
14400 / 15329
14500 / 15329
14600 / 15329
14700 / 15329
14800 / 15329
14900 / 15329
15000 / 15329
15100 / 15329
15200 / 15329
15300 / 153

In [ ]:
cluster.close()